In [1]:
import boto3
from botocore.exceptions import ClientError
import os

In [2]:
# Configuration
BUCKET_NAME = "data-ingestion-queue-bucket"  # Replace with your bucket name

# Create S3 client
s3_client = boto3.client("s3")


In [3]:
# List available buckets (to verify credentials work)
try:
    response = s3_client.list_buckets()
    print("Available buckets:")
    for bucket in response["Buckets"]:
        print(f"  - {bucket['Name']}")
except ClientError as e:
    print(f"Error listing buckets: {e}")


Available buckets:
  - data-ingestion-queue-bucket


In [4]:
def upload_file_to_s3(file_path: str, bucket: str, object_name: str = None) -> bool:
    """
    Upload a file to an S3 bucket.
    
    Args:
        file_path: Path to file to upload
        bucket: Target bucket name
        object_name: S3 object name. If not specified, file_path basename is used
    
    Returns:
        True if upload successful, False otherwise
    """
    if object_name is None:
        object_name = os.path.basename(file_path)
    
    try:
        s3_client.upload_file(file_path, bucket, object_name)
        print(f"✓ Successfully uploaded '{file_path}' to 's3://{bucket}/{object_name}'")
        return True
    except ClientError as e:
        print(f"✗ Upload failed: {e}")
        return False


In [5]:
test_file_path = "../data/uploads/test.pdf"
# Upload the test file to S3
upload_file_to_s3(test_file_path, BUCKET_NAME)


✓ Successfully uploaded '../data/uploads/test.pdf' to 's3://data-ingestion-queue-bucket/test.pdf'


True

In [6]:
# Verify upload by listing objects in bucket
try:
    response = s3_client.list_objects_v2(Bucket=BUCKET_NAME, MaxKeys=10)
    print(f"Objects in bucket '{BUCKET_NAME}':")
    if "Contents" in response:
        for obj in response["Contents"]:
            print(f"  - {obj['Key']} ({obj['Size']} bytes)")
    else:
        print("  (bucket is empty)")
except ClientError as e:
    print(f"Error listing objects: {e}")


Objects in bucket 'data-ingestion-queue-bucket':
  - test.pdf (552558 bytes)
